In [67]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import matplotlib.image as img
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

In [87]:
# Create a Colour Dictionary to use in the visuals

colours_dict ={
    'bug': '#A6B91A',
    'dark': '#705746',
    'dragon': '#6F35FC',
    'electric': '#F7D02C',
    'fairy':'#D685AD',
    'fighting': '#C22E28',
    'fire':'#EE8130',
    'ghost': '#735797',
    'grass':'#7AC74C',
    'ground':'#E2BF65',
    'ice':'#96D9D6',
    'normal':'#A8A77A',
    'poison':'#A33EA1',
    'psychic':'#F95587',
    'rock':'#B6A136',
    'steel':'#B7B7CE',
    'water':'#6390F0'
}

In [3]:
Name = []
Primary_Type = []
Weight = []
Height = []
HP = []
Speed = []
Attack = []
Defense = []
Experience = []

for x in range(1,252):
    url = "https://pokeapi.co/api/v2/pokemon/" + str(x)
    res = requests.get(url)
    Name.append(res.json()['name'])
    Primary_Type.append(res.json()['types'][0]['type']['name'])
    Weight.append(res.json()['weight'])
    Height.append(res.json()['height'])
    HP.append(res.json()['stats'][0]['base_stat'])
    Speed.append(res.json()['stats'][5]['base_stat'])
    Defense.append(res.json()['stats'][2]['base_stat'])
    Attack.append(res.json()['stats'][1]['base_stat'])
    Experience.append(res.json()['base_experience'])
    

In [41]:
Poke_Dict = {
    'Pokemon': Name,
    'Type': Primary_Type,
    'Weight': Weight,
    'Height': Height,
    'Experience':Experience,
    'HP': HP,
    'Speed': Speed,
    'Attack': Attack,
    'Defense': Defense
}
Pokemon_DF = pd.DataFrame(Poke_Dict)
Pokemon_DF.index += 1 

In [65]:
# Bar chart Showing Mean HP by Pokemon Type

HP_Means = round(Pokemon_DF.groupby(by = 'Type').mean().reset_index()[['Type','HP']],2)

HPBar = px.bar(
    HP_Means.sort_values(by = 'HP',ascending = False)[0:5],
    x = 'Type',
    y = 'HP',
    color='Type',
    hover_name = 'Type',
    color_discrete_map = colours_dict
)
HPBar.update_layout(
    showlegend=False,
    title={
    'text': 'Mean HP by Pokemon Type',
    'y':.923,
    'x':0.5,
    'xanchor': 'center',
    'yanchor': 'top'}
)

HPBar.show()

In [42]:

scatter = px.scatter(
    Pokemon_DF,
    x = 'Experience',
    y = 'HP',
    trendline= "ols",
    hover_name = 'Pokemon'
)

scatter.show()

In [40]:
# Correlation Matrix to see which variables affect eachother.

Pokemon_Corr = Pokemon_DF.corr()
x = list(Pokemon_Corr.columns)
y = list(Pokemon_Corr.index)
z = np.array(Pokemon_Corr)

fig = ff.create_annotated_heatmap(
    z,
    x = x,
    y = y,
    annotation_text = np.around(z, decimals=2),
    hoverinfo='z',
    colorscale='Viridis'
)
fig.update_layout(
    title={
    'text': 'Pokemon Attribute Correlation Matrix',
    'y':.9,
    'x':0.5,
    'xanchor': 'center',
    'yanchor': 'top'}
)
fig.show()

In [19]:
Pokemon_Corr = Pokemon_DF.corr()

,Weight,Height,Experience,HP,Speed,Attack,Defense
Weight,1.000000,0.691014,0.445351,0.394460,0.116086,0.457429,0.474933
Height,0.691014,1.000000,0.381505,0.271339,0.200649,0.334597,0.439712
Experience,0.445351,0.381505,1.000000,0.707964,0.401584,0.437339,0.364206
HP,0.394460,0.271339,0.707964,1.000000,0.025721,0.245700,0.122550
Speed,0.116086,0.200649,0.401584,0.025721,1.000000,0.277380,-0.032161
Attack,0.457429,0.334597,0.437339,0.245700,0.277380,1.000000,0.443174
Defense,0.474933,0.439712,0.364206,0.122550,-0.032161,0.443174,1.000000


In [43]:

scatter = px.scatter(
    Pokemon_DF.sort_values(by = 'Experience',ascending = False)[2:len(Pokemon_DF)-2],
    x = 'Experience',
    y = 'HP',
    trendline= "ols",
    hover_name = 'Pokemon'
)

scatter.show()

In [56]:
Pokemon_DF.sort_values(by = 'Attack',ascending = False)[0:5][['Pokemon','Type','Attack']]

,Pokemon,Type,Attack
248,tyranitar,rock,134
149,dragonite,dragon,134
136,flareon,fire,130
250,ho-oh,fire,130
68,machamp,fighting,130


In [49]:
Pokemon_DF['attack_Exp_Ratio'] = Pokemon_DF['Attack']/Pokemon_DF['Experience']

In [52]:
Pokemon_DF.sort_values(by = 'attack_Exp_Ratio', ascending = False)

,Pokemon,Type,Weight,Height,Experience,HP,Speed,Attack,Defense,attack_Exp_Ratio
98,krabby,water,65,4,65,30,50,105,90,1.615385
84,doduo,normal,392,14,62,35,75,85,45,1.370968
209,snubbull,fairy,78,6,60,60,30,80,50,1.333333
74,geodude,rock,200,4,60,40,20,80,100,1.333333
56,mankey,fighting,280,5,61,40,70,80,35,1.311475
...,...,...,...,...,...,...,...,...,...,...
12,butterfree,bug,320,11,198,60,70,45,50,0.227273
65,alakazam,psychic,480,15,250,55,120,50,45,0.200000
213,shuckle,bug,205,6,177,20,5,10,230,0.056497
242,blissey,normal,468,15,635,255,55,10,10,0.015748


In [96]:
# Bar chart Showing Mean HP by Pokemon Type

Defense_Means = round(Pokemon_DF.groupby(by = 'Type').mean().reset_index()[['Type','Defense']],2)

DefenseBar = px.bar(
    Defense_Means.sort_values(by = 'Defense',ascending = False)[0:5],
    x = 'Type',
    y = 'Defense',
    color='Type',
    hover_name = 'Type',
    color_discrete_map = colours_dict
)
DefenseBar.update_layout(
    showlegend=False,
    title={
    'text': 'Mean HP by Pokemon Type',
    'y':.955,
    'x':0.5,
    'xanchor': 'center',
    'yanchor': 'top'}
)

DefenseBar.show()

In [69]:
sub_plots = make_subplots(rows = 1, cols = 2)
sub_plots.add_trace(HPBar,row=1, col=1)
sub_plots.add_trace(DefenseBar,row=1, col=1)

ValueError: 
    Invalid element(s) received for the 'data' property of 
        Invalid elements include: [Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': '<b>%{hovertext}</b><br><br>Type=%{x}<br>HP=%{y}<extra></extra>',
              'hovertext': array(['normal'], dtype=object),
              'legendgroup': 'normal',
              'marker': {'color': '#A8A77A', 'pattern': {'shape': ''}},
              'name': 'normal',
              'offsetgroup': 'normal',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['normal'], dtype=object),
              'xaxis': 'x',
              'y': array([82.14]),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': '<b>%{hovertext}</b><br><br>Type=%{x}<br>HP=%{y}<extra></extra>',
              'hovertext': array(['psychic'], dtype=object),
              'legendgroup'

In [95]:
Poke_Count = Pokemon_DF[['Pokemon','Type']].groupby(by = 'Type').count().reset_index().sort_values(by = 'Pokemon',ascending = False)

# Bar chart Showing Mean HP by Pokemon Type

CountBar = px.bar(
    Poke_Count,
    x = 'Type',
    y = 'Pokemon',
    color='Type',
    hover_name = 'Type',
    color_discrete_map = colours_dict
)
CountBar.update_layout(
    showlegend=False,
    title={
    'text': 'Count of Pokemon by Type',
    'y':.9599,
    'x':0.5,
    'xanchor': 'center',
    'yanchor': 'top'}
)

CountBar.show()
CountBar.write_html("output.html", full_html=False, include_plotlyjs='cdn')